In [1]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("zaheenhamidani/ultimate-spotify-tracks-db")

print("Path to dataset files:", path)

df = pd.read_csv(path + "/SpotifyFeatures.csv")
df.head()


100%|██████████| 15.5M/15.5M [00:00<00:00, 55.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zaheenhamidani/ultimate-spotify-tracks-db/versions/3


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [2]:
features = df[['danceability', 'energy',  'loudness',
               'speechiness', 'acousticness', 'instrumentalness',
               'liveness', 'valence', 'tempo']]
features

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.389,0.910,-1.828,0.0525,0.61100,0.000000,0.3460,0.814,166.969
1,0.590,0.737,-5.559,0.0868,0.24600,0.000000,0.1510,0.816,174.003
2,0.663,0.131,-13.879,0.0362,0.95200,0.000000,0.1030,0.368,99.488
3,0.240,0.326,-12.178,0.0395,0.70300,0.000000,0.0985,0.227,171.758
4,0.331,0.225,-21.150,0.0456,0.95000,0.123000,0.2020,0.390,140.576
...,...,...,...,...,...,...,...,...,...
232720,0.687,0.714,-10.626,0.0316,0.00384,0.544000,0.0845,0.962,115.542
232721,0.785,0.683,-6.944,0.0337,0.03290,0.000880,0.2370,0.969,113.830
232722,0.517,0.419,-8.282,0.1480,0.90100,0.000000,0.0945,0.813,84.135
232723,0.745,0.704,-7.137,0.1460,0.26200,0.000000,0.3330,0.489,100.031


In [3]:
# normalization

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


In [4]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
model.fit(scaled_features)


NearestNeighbors(algorithm='ball_tree', n_neighbors=6)

In [5]:
def recommend(song_name):
    index = df[df['track_name'].str.lower() == song_name.lower()].index[0]
    distances, indices = model.kneighbors([scaled_features[index]])

    print(f"Recommendations for '{song_name}':\n")
    for i in indices[0][1:]:
        print(df.iloc[i]['track_name'], '-', df.iloc[i]['artist_name'])


In [6]:
recommend("Shape of You")


Recommendations for 'Shape of You':

Looking Out for You - Joy Again
Guayaquil - Don Omar
Every Little Thing - Russell Dickerson
Every Little Thing - Russell Dickerson
Every Little Thing - Russell Dickerson


In [7]:
from sklearn.cluster import KMeans

# Assuming 'scaled_features' is already created (StandardScaler used)
kmeans = KMeans(n_clusters=10, random_state=42)
df['cluster'] = kmeans.fit_predict(scaled_features)


In [8]:
def recommend_kmeans(song_name):
    idx = df[df['track_name'].str.lower() == song_name.lower()].index[0]
    cluster_id = df.loc[idx, 'cluster']

    similar_songs = df[df['cluster'] == cluster_id]
    print(f"Songs similar to '{song_name}':\n")
    print(similar_songs[['track_name', 'artist_name']].sample(5))


In [9]:
recommend_kmeans("shape of you")

Songs similar to 'shape of you':

                                           track_name          artist_name
109782                              Floss In The Bank                 Tyga
19756   Me Against the Music - LP Version / Video Mix       Britney Spears
62590                             Pretend You Love Me  Sonny & The Sunsets
181341                                        Clarity    Harvey Sutherland
148457             Troubled Land (feat. James Junior)          Speedometer
